In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import scipy
import threading
import python_speech_features
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import IPython.display as ipd
import math
%matplotlib inline
import librosa
import pickle
import pywt
from pywt import dwt, wavedec
from os import listdir
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Dropout
from os.path import isfile, join

In [ ]:
trainDir = "free-spoken-digit-dataset-master/recordings/"
# testDir = "test/"
trainFiles = [f for f in listdir(trainDir) if isfile(join(trainDir, f))]
# testFiles = [f for f in listdir(testDir) if isfile(join(testDir, f))]
# print(len(trainFiles))
# print(len(testFiles))
# metaFile = "FSDKaggle2018.meta/train_post_competition.csv"
# meta = pd.read_csv(metaFile)

In [ ]:
def returnMelCoeff(signal, sr):
    return librosa.feature.mfcc(signal, sr)

def windowHann(signal, fr, windowSize):
    window = get_window("hann", windowSize)
    windowed = signal * window
    return windowed

def padSignal(signal, sr, windowSize, hopSize):
    signalLength = len(signal)
    numZeros = windowSize - (signalLength - math.floor(signalLength/hopSize)*hopSize);
    return np.concatenate((signal, np.zeros(numZeros)), axis=None)

# def process(signal, sr, windowSize, hopSize):
#     origLength = len(signal)
#     signal = padSignal(signal, sr, windowSize, hopSize)
#     mfccArray = []
#     for i in range(0, origLength, hopSize):
#         toProcess = windowHann(signal[i: i+windowSize], sr, windowSize)
#         tempArr = []
#         a, d5, d4, d3, d2, d1 = wavedec(toProcess, 'db32', level=levels)
#         mfccs5 = python_speech_features.mfcc(d5, sr, winlen=(len(d5)/sr), nfft = len(d5), winstep=(len(d5)/sr), numcep=1)
#         mfccs4 = python_speech_features.mfcc(d4, sr, winlen=(len(d4)/sr), nfft = len(d4), winstep=(len(d4)/sr), numcep=1)
#         mfccs3 = python_speech_features.mfcc(d3, sr, winlen=(len(d3)/sr), nfft = len(d3), winstep=(len(d3)/sr), numcep=2)
#         mfccs2 = python_speech_features.mfcc(d2, sr, winlen=(len(d2)/sr), nfft = len(d2), winstep=(len(d2)/sr), numcep=3)
#         mfccs1 = python_speech_features.mfcc(d1, sr, winlen=(len(d1)/sr), nfft = len(d1), winstep=(len(d1)/sr), numcep=5)
#         mfccsa = python_speech_features.mfcc(a, sr, winlen=(len(a)/sr), nfft = len(a), winstep=(len(a)/sr), numcep=1)
#         tempArr.append(mfccs5[0])
#         tempArr.append(mfccs4[0])
#         tempArr.append(mfccs3[0])
#         tempArr.append(mfccs2[0])
#         tempArr.append(mfccs1[0])
#         tempArr.append(mfccsa[0])
#         mfccArray.append([p for d in tempArr for p in d])

#     return mfccArray

windowSize = 1235
hopSize = 1102
levels = 5

fName = os.path.join(trainDir, trainFiles[1])
signal, sr = librosa.load(fName)
mfccArray = process(signal, sr, windowSize, hopSize)


with open('dumpfilewaveletsMNIST.pickle', 'wb') as handle:
    for i in trainFiles:
        fName = os.path.join(trainDir, i)
        signal, sr = librosa.load(fName)
        mfccArray = process(signal, sr, windowSize, hopSize)
        a = {i: mfccArray}
        pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print("Done with: ", i)

In [ ]:
levels = [3, 4, 5, 6]
wavelets = ['bior2.4']
windowSize = 512
hopSize = 128

def windowHann(signal, fr, windowSize):
    window = get_window("hann", windowSize)
    windowed = signal * window
    return windowed

def padSignal(signal, sr, windowSize, hopSize):
    signalLength = len(signal)
    numZeros = windowSize - (signalLength - math.floor(signalLength/hopSize)*hopSize);
    return np.concatenate((signal, np.zeros(numZeros)), axis=None)

def processWaveletMFCCs(signal, sr, windowSize, hopSize, lv, wavelet):
    origLength = len(signal)
    signal = padSignal(signal, sr, windowSize, hopSize)
    mfccArray = np.empty((math.ceil(origLength/hopSize), lv+1, 20))
    block = 0
    for i in range(0, origLength, hopSize):
        toProcess = windowHann(signal[i: i+windowSize], sr, windowSize)
        dec = wavedec(toProcess, wavelet, level=lv)
        for j in range(len(dec)):
            mfccArray[block, j, :] = python_speech_features.mfcc(dec[j], sr, winlen=(len(dec[j])/sr), nfft = len(dec[j]), winstep=(len(dec[j])/sr), numcep=20)[0]
        block+=1
    return mfccArray

# testName = trainFiles[2]
# fName = os.path.join(trainDir, testName)
# signal, sr = librosa.load(fName)
# print(len(signal))
# mfccArray = processWaveletMFCCs(signal, sr, windowSize, hopSize, levels[0], wavelets[0])
# print(np.shape(mfccArray))


for wavelet in wavelets:
    for lv in levels:
        fileName = "MNIST_"+(wavelet).replace('.','') + "_"+str(lv)+".pickle"
        with open(fileName, 'wb') as handle:
            for i in trainFiles:
                fName = os.path.join(trainDir, i)
                signal, sr = librosa.load(fName)
                mfccArray = processWaveletMFCCs(signal, sr, windowSize, hopSize, lv, wavelet)
                a = {i: mfccArray}
                pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
objects = []
with (open("dumpfilewaveletsMNIST.pickle", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

In [ ]:
import pywt
pywt.wavelist('bior')

In [1]:
lab = {}

for i in range(10):
    tag = df["filepath"][i].split("/")[1]
    y_label = np.zeros((10))
    y_label[i] = 1
    lab[tag] = y_label

NameError: name 'df' is not defined